# Example of Running `tweakwcs` to Align JWST images

***
## About this notebook
**Authors:** Mihai Cara (STScI), Clare Shanahan (STScI)
<br>**Updated On:** 12/05/2018

This notebooks illustrates a basic workflow of how to use `tweakwcs` to align JWST images.

***
## Imports

In [ ]:
from copy import deepcopy

from jwst.datamodels import ImageModel
import matplotlib.pyplot as plt 
import numpy as np
from photutils import DAOStarFinder, detect_threshold
from tweakwcs import tweak_image_wcs

# 'matplotlib inline' for displaying plots nicely in this notebook.
%matplotlib inline 

***
## Download & Load Data

The data for this example are hosted on box and are available to anyone with the static link below. The cells below, when executed, will download these files to your machine at the paths stored as strings in variables `example_file_1` and `example_file_2`.

In [ ]:
from astropy.utils.data import download_file

url1 = 'https://stsci.box.com/shared/static/3z78bjae14pj6nq3plvauukpmw1li1hc.fits'
url2 = 'https://stsci.box.com/shared/static/d301ydkzxqfnd559rl4r8qts5hw2x1is.fits'
example_file_1 = download_file(url1, cache=True)
example_file_2 = download_file(url2, cache=True)

Load JWST ImageModels from downloaded files and inspect exposures.

In [ ]:
im1 = ImageModel(example_file_1)
im2 = ImageModel(example_file_2)

f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8,4))
ax1.imshow(im1.data, origin = 'lower', vmin=-0.5, vmax=0.5, cmap='Greys_r')
ax1.set_title('Image 1')
ax2.imshow(im2.data, origin='lower', vmin=-0.5, vmax=0.5, cmap='Greys_r')
ax2.set_title('Image 2')
pass;

We are working with simulated NIRCam Observations, two dithered exposures of the same field. Because these datasets are already accurately aligned, we will introduce an artificial WCS error to show how `tweakwcs` can account for misalignment. To do this, we will simply take the GWCS from `im1` and assign it to `im2`, mimicking a scenario where pointing information for `im2` is incorrect and so pixel positions will map incorrectly to the sky.

Here we will also save a copy of the original GWCS for comparison after the alignment process

In [ ]:
wcs_im1 = im1.meta.wcs
wcs_im2 = deepcopy(wcs_im1)

original_wcs_im1 = deepcopy(wcs_im1)
original_wcs_im2 = deepcopy(wcs_im1)

***
## Typical Workflow to Align Two or More Images

We will align the two images (ImageModels loaded above) using the most basic workflow. 

This process consists of the following steps.

    1. Detect sources in each image and create source catalogs. 
    2. Insert these catalogs in the ImageModels in 'meta', where tweakwcs will look for them. 
    3. Inspect the output to verify the quality of the alignment.
    4. When satisfied, save the aligned image. 


### Create Source Catalogs

`tweakwcs` does not create source catalogs used for matching internally, they must be passed in. This increases the flexibility of use - users may generate these catalogs any way they like (segmentation maps, psf fitting, or `DAOStarFinder` as used here, for example). Users may also apply selection criteria to source catalogs before passing them in, such as selecting only point sources, or those in certain magnitude range. In this example, we simply select all sources detected with `DAOStarFinder` that are over a certain threshold. 
Images will be aligned based on sources common between the frames. Source catalogs are generated for each image using the `DAOStarFinder` algorithm in `photutils`. Once the sources have been detected, the resulting catalog will be added to each ImageModel in `meta`. 

In [ ]:
for catno, im in enumerate([im1, im2]):  # iterate over images to create and insert catalogs into imagemodel
    threshold = detect_threshold(im.data, snr=150.)[0, 0]
    daofind = DAOStarFinder(fwhm=5, threshold=threshold)
    cat = daofind(im.data)
    cat.rename_column('xcentroid', 'x')
    cat.rename_column('ycentroid', 'y')
    cat.meta['name'] = 'im{:d} sources'.format(catno)
    print(len(cat), 'sources detected in im{}'.format(catno+1))
    im.meta['catalog'] = cat 
cat1 = im1.meta['catalog']
cat2 = im2.meta['catalog']

####  Plot detected sources

In [ ]:
f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize = (12, 6)); f.suptitle('All Detected sources')
ax1.imshow(im1.data, origin='lower', vmin=-0.5, vmax=0.5, cmap='Greys_r')
ax1.scatter(cat1['x'], cat1['y'], facecolor='None', edgecolor='r')
ax1.set_xlim(0, 2000)
ax1.set_ylim(0, 2000)
ax1.set_title('Image 1')
ax2.imshow(im2.data, origin='lower', vmin=-0.5, vmax=0.5, cmap='Greys_r')
ax2.scatter(cat2['x'],cat2['y'], facecolor='None', edgecolor='r')
ax2.set_xlim(0, 2000)
ax2.set_ylim(0, 2000)
ax2.set_title('Image 2'); pass;

## Align Images with tweakwcs

With source catalogs created and added to each image model, they can now be aligned with `tweak_image_wcs`. This function modifies the ImageModel in memory to adjust GWCS information. By default, a reference catalog will be used from the first image passed in. In this case, the reference catalog is in im1, so im2 will be aligned to im1 and only the WCS information for im2 will be adjusted.

Users have the option to supply an external reference catalog to align each image to as well - if that was done in this case, the WCS both im1 and im2 would be aligned to that catalog and so both files would be modified.  

In [ ]:
tweak_image_wcs([im1, im2])

`tweakwcs` inserts a correction into the full transformation pipeline for each image being aligned- in this example, im2 - labeled 'tangent-plane linear correction'.

In [ ]:
print(im2.meta.wcs)

## Inspecting quality of alignment

A typical workflow may entail several attempts at alignment before it is successful. Verifying the quality of an alignment attempt involves comparing the projected position of sources matched between input catalogs on the sky with each input image's WCS information, and verifying that they agree well and there are no systematics indicating a poor alignment. Statistics about the fit, like the RMS, are also useful to look at. 

`tweak_image_wcs` updates file objects in memory. For images being aligned, it will update the GWCS in `meta.wcs`, and insert fit and match information in `meta` (or `meta.info`, if present), useful for diagnosing the quality of the alignment.

Note that the alignment info must be deleted before the file is saved in order to avoid schema validation issues, so
in the cell below we will load it into a variable then remove it from the file object.

In [ ]:
twi2 = im2.meta.get('tweakwcs_info', None).instance
# delete info from meta after storing in variable in order to avoid schema validation issues
del im2.meta['tweakwcs_info']

Let's inspect what information about the `tweak_image_wcs` run this attribute contains. Remember that im2 was aligned to im1, so only im2 was modified.

In [ ]:
print(twi2)

We have information about which objects were matched between the two catalogs and therefore used for the alignment. The indices of these sources in the reference catalog from im1 are contained in `matched_ref_idx`, and the indices of these same sources in im2's catalog are stored in `matched_input_idx`.

It is useful to compare common sources between images to inspect the quality of the alignment before and after image re-alignment. If these images are well aligned, then when we project the same source from each image onto the sky with the GWCS transformation, they should match and the residuals should be minimal and mostly random. 

First, we will calculate and plot the sky positions of these matched sources using each image's **original** GWCS, to show the misalignment we started with.

In [ ]:
# select matched source positions from input catalog, using known indices
matched_tab1_x_y = [(cat1['x'][i], cat1['y'][i]) for i in twi2['matched_ref_idx']]
matched_tab2_x_y = [(cat2['x'][i], cat2['y'][i]) for i in twi2['matched_input_idx']]

# transform matched sources pixel positions to sky using ORIGINAL GWCS 
im1_ra, im1_dec = original_wcs_im1([x[0] for x in matched_tab1_x_y], [x[1] for x in matched_tab1_x_y])
im2_ra, im2_dec = original_wcs_im2([x[0] for x in matched_tab2_x_y], [x[1] for x in matched_tab2_x_y])

q = plt.quiver(im1_ra, im1_dec, -1.*(im1_ra-im2_ra), im1_dec-im2_dec)
plt.xlim(plt.xlim()[::-1])
plt.title('residuals, BEFORE alignment')
plt.xlabel('ra (reference image)')
plt.ylabel('dec (reference image)'); pass;
print('median absolute offset BEFORE alignment, RA and Dec (arcsec): ', 
      np.median(np.abs(im2_ra-im1_ra)*3600.), 
      np.median(np.abs(im2_dec-im1_dec)*3600.))

The systematic nature of these residuals, and the large offset of several arcseconds, indicates a poor alignment to begin with. Let's see if running `tweakwcs` improved this alignment so that sources projected with GWCS from image 1 better match the sky position of the same sources projected with the GWCS from image 2. To do this, we will do the same procedure as the cell above but use the tweaked WCS for im2 to convert pixel to sky positions for the common sources. We can also print the RMS of the fit to the screen.

In [ ]:
# transform matched sources pixel positions to sky using NEW GWCS for im2
im2_ra, im2_dec = im2.meta.wcs([x[0] for x in matched_tab2_x_y], [x[1] for x in matched_tab2_x_y])

q = plt.quiver(im1_ra, im1_dec, -1.*(im1_ra-im2_ra), im1_dec-im2_dec)
plt.xlim(plt.xlim()[::-1])
plt.title('residuals, AFTER alignment')
plt.xlabel('ra (reference image)')
plt.ylabel('dec (reference image)'); pass;
print('x, y fit RMS (pixels)', twi2['rms'])
print('median absolute offset AFTER alignment, RA and Dec (arcsec): ', 
      np.median(np.abs(im2_ra-im1_ra)*3600.), 
      np.median(np.abs(im2_dec-im1_dec)*3600.))

This is what the offset between common sources projected on the sky from im1 and im2 look like after alignment with `tweakwcs` - keep in mind that the arrow sizes are normalized. The randomness is an indicator of a good fit, and the residuals are on the order of a tiny fraction of an arcsecond. 

### Save Aligned Image.

Because TweakWCS works in memory, we have to explicitly save the tweaked GWCS to file once we are satisfied with the alignment. This can be done with the ImageModel.write method.

In [ ]:
im2.write('im2_aligned.fits')